In [1]:
import numpy as np
import pandas as pd
from functools import partial
import matplotlib.pyplot as plt

# for dataset preprocessing
from  sklearn.preprocessing  import  StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

#for ANN and optimisator
from valid import  validation, accuracy
from NeuroNet import NeuroNet
from my_PSO import ParticleSwarm
from jaya_for_ann import Jaya

In [2]:
from ucimlrepo import fetch_ucirepo 
  
glass_identification = fetch_ucirepo(id=42) 
  
X = glass_identification.data.features.to_numpy() 
u = glass_identification.data.targets.to_numpy() 
y = u.T[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

scaler = StandardScaler()  
encoder = OneHotEncoder()
X_train =  scaler.fit_transform(X_train.astype(np.float64)) 
X_test  =  scaler.fit_transform(X_test.astype(np.float64)) 

y_train = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test = encoder.transform(y_test.reshape(-1, 1)).toarray()


n_feat  = len(X_test[0])
m_out   = 6
print('X_train:' , len(X_train))
print('n_feat:', n_feat)


X_train: 160
n_feat: 9


In [3]:
num_iter = 500 # количество итераций
new_start = 1 # число симуляций 

In [4]:
#PSO
neuro_pso = NeuroNet(n_feat=n_feat, m_out=m_out, swarmsize=40, dw=-1, up=1, hid=18, lmbd=0.00001) 
pso_bestGlobal = []
all_best_scores_pso = []
for k in range(new_start):
    p_swarm = neuro_pso.initialize_swarm()
    f_cost_pso = partial(neuro_pso.err_for_each_particle, X_train, y_train)
    pso = ParticleSwarm(x_max=np.inf, v_max=0.1, swarm=p_swarm, cost_f=f_cost_pso, dimension=neuro_pso.dimension, swarmsize=neuro_pso.swarmsize, alpha=2, beta=1, gamma=0.5)
    best_scores_pso = []
    i = 0
    while pso.best > 1e-6 and i < num_iter:
        pso.rate_particles()
        best_scores_pso.append(pso.best) #минимальная ошибка в рое на каждой итерации
        i = i + 1
    pso_bestGlobal.append(pso.bestGlobal) # положение частицы в котором достигается минимальная ошибка после всех итераций
    all_best_scores_pso.append(best_scores_pso) # список минимальных ошибок в рое  на каждой симуляции. найти их mean, median, std, min

In [5]:
#Jaya
neuro_jaya = NeuroNet(n_feat=n_feat, m_out=m_out, swarmsize=40, dw=-1, up=1, hid=18, lmbd=0.0000001) 
jaya_best = []
all_best_scores_jaya = []
for k in range(new_start):
    p_population = neuro_jaya.initialize_swarm()
    f_cost_jaya = partial(neuro_jaya.err_for_each_particle, X_train, y_train)
    jaya = Jaya(domain=np.inf, population=p_population, cost_f=f_cost_jaya, obj=neuro_jaya)
    best_scores_jaya = []
    i = 0
    while jaya.best_value > 1e-4 and i < num_iter:
        jaya.update()
        best_scores_jaya.append(jaya.best_value)
        i = i + 1
    jaya_best.append(jaya.best)
    all_best_scores_jaya.append(best_scores_jaya)

In [6]:
#PSO-Jaya
pso_jaya_best = []
all_best_scores_pso_jaya = []
for k in range(new_start):
    neuro_pso_jaya = NeuroNet(n_feat=n_feat, m_out=m_out, swarmsize=40, dw=-1, up=1, hid=18, lmbd=0.0000001) 
    f_cost_pso_jy = partial(neuro_pso_jaya.err_for_each_particle, X_train, y_train)
    p_swarm_pop = neuro_pso_jaya.initialize_swarm()
    pso_jaya =  ParticleSwarm(x_max=np.inf, v_max=0.1, swarm=p_swarm_pop, cost_f=f_cost_pso_jy, dimension=neuro_pso_jaya.dimension, swarmsize=neuro_pso_jaya.swarmsize, alpha=2, beta=1, gamma=0.5)
    best_scores_pso_jaya = [pso_jaya.best]
    for i in range(num_iter):
        pso_jaya.rate_particles()
        if pso_jaya.best >= best_scores_pso_jaya[-1]:
            jy = Jaya(domain=np.inf, cost_f=pso_jaya.cost_f, population=pso_jaya.swarm, obj=pso_jaya)
            for j in range(3):
                jy.update()
            pso_jaya.bestGlobal = jy.best
        best_scores_pso_jaya.append(pso_jaya.best) 
    pso_jaya_best.append(pso_jaya.bestGlobal)
    all_best_scores_pso_jaya.append(best_scores_pso_jaya)

In [7]:
valid_pso = accuracy(neuro_pso, X_train, X_test, y_train, y_test, pso_bestGlobal)
valid_jaya = accuracy(neuro_jaya, X_train, X_test, y_train, y_test, jaya_best)
valid_pso_jaya = accuracy(neuro_pso_jaya, X_train, X_test, y_train, y_test, pso_jaya_best)

print("PSO:",      valid_pso.T[0].mean(),     valid_pso.T[1].mean())
print("Jaya:",     valid_jaya.T[0].mean(),     valid_jaya.T[1].mean())
print("PSO-Jaya:", valid_pso_jaya.T[0].mean(), valid_pso_jaya.T[1].mean())

PSO: 69.375 70.37037037037037
Jaya: 63.125 70.37037037037037
PSO-Jaya: 72.5 66.66666666666667


In [ ]:
def all_best_scores(all_best_scores):
    res_simul = []
    for el in all_best_scores.T:
        res_simul.append(el.mean())
    return res_simul

In [ ]:
all_best_scores_pso      = np.asarray(all_best_scores_pso)
all_best_scores_jaya     = np.asarray(all_best_scores_jaya)
all_best_scores_pso_jaya = np.asarray(all_best_scores_pso_jaya)

res_simul_pso      = all_best_scores(all_best_scores_pso)
res_simul_jaya     = all_best_scores(all_best_scores_jaya)
res_simul_pso_jaya = all_best_scores(all_best_scores_pso_jaya)

In [ ]:
def print_statistical_analysis(all_best_scores):
    print("mean:", np.mean(all_best_scores.T[-1]))
    print("median:", np.median(all_best_scores.T[-1]))
    print("std:", np.std(all_best_scores.T[-1]))
    print("min", np.min(all_best_scores.T[-1]))

print("--pso--")
print_statistical_analysis(all_best_scores_pso)
print("--jaya--")
print_statistical_analysis(all_best_scores_jaya)
print("--pso_jaya--")
print_statistical_analysis(all_best_scores_pso_jaya)

In [ ]:
res_simul_pso = np.asarray(res_simul_pso)
n = np.arange(res_simul_pso.shape[0])
plt.plot(n, res_simul_pso, "--r", label="pso")
plt.plot(n, res_simul_jaya, "b", label="jaya")
plt.plot(n, res_simul_pso_jaya[1:501], ":g", label="pso_jaya")


plt.xlabel("iteration")
plt.ylabel("error")
plt.legend()
plt.savefig("pin02.png", dpi=600)
plt.grid('True')
plt.semilogy()

In [ ]:
df = pd.DataFrame({'PSO': all_best_scores_pso.T[-1], 'Jaya': all_best_scores_jaya.T[-1], 'pso_jaya': all_best_scores_pso_jaya.T[-1]})

#create boxplot by group
plt.boxplot(df, labels=['pso', 'jaya', 'pso-jaya'])
plt.xlabel("algorithm")
plt.ylabel("error")
plt.semilogy()